In [1]:
from easydict import EasyDict as edict

### Hyperparameters
Here, we build a dictionary including the hyperparameters that we will use in training and testing. In all these models, we will use "Adam" as the optimizer.

In [2]:
config = edict()
config.TRAIN = edict()
config.VALID = edict()

## some parameters for Adam
config.TRAIN.batch_size = 4 # depends on GPU's memory, use 8 or 16 for faster training if allowed
config.TRAIN.lr_init = 1e-4 # the initial learning rate
config.TRAIN.beta1 = 0.9 # The exponential decay rate for the 1st moment estimates

## parameters for initializing Generator
config.TRAIN.n_epoch_init = 50 # use 100 for better performance
    
## parameters for adversarial learning (SRGAN)
config.TRAIN.n_epoch = 50 # use 2000 for better performance if allowed
config.TRAIN.lr_decay = 0.1 # define the decay of learning rate here
config.TRAIN.decay_every = int(config.TRAIN.n_epoch / 2) # define the frequency of learning rate decay here

## training set location
config.TRAIN.hr_img_path = '../srgan/DIV2K_train_HR/'
config.TRAIN.lr_img_path = '../srgan/DIV2K_train_LR_bicubic/X4/'

## test set location
config.VALID.hr_img_path = '../srgan/DIV2K_valid_HR/'
config.VALID.lr_img_path = '../srgan/DIV2K_valid_LR_bicubic/X4/'

In [3]:
import os
import time
import random
import skimage
import numpy as np
import scipy, multiprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import (Input, Conv2d, BatchNorm2d, Elementwise, SubpixelConv2d, Flatten, Dense)
from tensorlayer.models import Model
from PIL import Image

### Models
In this part, we define three functions of building models for the later need. And we apply the tensorlayer library to build all the models.
- Generator for training
- Generator for testing
- Discriminator for training

In [4]:
def get_G(input_shape):
    w_init = tf.random_normal_initializer(stddev=0.02) # the initializer for weight matrix
    g_init = tf.random_normal_initializer(1., 0.02) # initializer for initializing gamma in batch normalization

    nin = Input(input_shape)
    n = Conv2d(64, (3, 3), (1, 1), act=tf.nn.relu, padding='SAME', W_init=w_init)(nin)
    temp = n

    # B residual blocks
    for i in range(16):
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
        nn = BatchNorm2d(act=tf.nn.relu, gamma_init=g_init)(nn)
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(nn)
        nn = BatchNorm2d(gamma_init=g_init)(nn)
        nn = Elementwise(tf.add)([n, nn])
        n = nn

    n = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(gamma_init=g_init)(n)
    n = Elementwise(tf.add)([n, temp])
    # B residual blacks end

    n = Conv2d(256, (3, 3), (1, 1), padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    n = Conv2d(256, (3, 3), (1, 1), act=None, padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    nn = Conv2d(3, (1, 1), (1, 1), act=tf.nn.tanh, padding='SAME', W_init=w_init)(n)
    G = Model(inputs=nin, outputs=nn, name="generator")
    return G

In [4]:
def get_Gval(input_shape):
    w_init = tf.random_normal_initializer(stddev=0.02)
    g_init = tf.random_normal_initializer(1., 0.02)

    nin = Input(input_shape)
    n = Conv2d(64, (3, 3), (1, 1), act=tf.nn.relu, padding='SAME', W_init=w_init)(nin)
    temp = n

    # B residual blocks
    for i in range(16):
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
        nn = BatchNorm2d(act=tf.nn.relu, gamma_init=g_init)(nn)
        nn = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(nn)
        nn = BatchNorm2d(gamma_init=g_init)(nn)
        nn = Elementwise(tf.add)([n, nn])
        n = nn

    n = Conv2d(64, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(gamma_init=g_init)(n)
    n = Elementwise(tf.add)([n, temp])
    # B residual blacks end

    n = Conv2d(256, (3, 3), (1, 1), padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    n = Conv2d(256, (3, 3), (1, 1), act=None, padding='SAME', W_init=w_init)(n)
    n = SubpixelConv2d(scale=2, n_out_channels=None, act=tf.nn.relu)(n)

    nn = Conv2d(3, (1, 1), (1, 1), act=tf.nn.tanh, padding='SAME', W_init=w_init)(n)
    Gval = Model(inputs=nin, outputs=nn)
    return Gval

In [6]:
def get_D(input_shape):
    w_init = tf.random_normal_initializer(stddev=0.02) # the initializer for weight matrix
    gamma_init = tf.random_normal_initializer(1., 0.02) # initializer for initializing gamma in batch normalization
    df_dim = 64
    lrelu = lambda x: tl.act.lrelu(x, 0.2)

    nin = Input(input_shape)
    n = Conv2d(df_dim, (4, 4), (2, 2), act=lrelu, padding='SAME', W_init=w_init)(nin)

    n = Conv2d(df_dim * 2, (4, 4), (2, 2), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 4, (4, 4), (2, 2), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 8, (4, 4), (2, 2), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 16, (4, 4), (2, 2), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 32, (4, 4), (2, 2), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 16, (1, 1), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 8, (1, 1), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    nn = BatchNorm2d(gamma_init=gamma_init)(n)

    n = Conv2d(df_dim * 2, (1, 1), (1, 1), padding='SAME', W_init=w_init, b_init=None)(nn)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 2, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(act=lrelu, gamma_init=gamma_init)(n)
    n = Conv2d(df_dim * 8, (3, 3), (1, 1), padding='SAME', W_init=w_init, b_init=None)(n)
    n = BatchNorm2d(gamma_init=gamma_init)(n)
    n = Elementwise(combine_fn=tf.add, act=lrelu)([n, nn])

    n = Flatten()(n)
    no = Dense(n_units=1, W_init=w_init)(n)
    D = Model(inputs=nin, outputs=no, name="discriminator")
    return D

### Training
Below is the main traininig process.

In [7]:
###====================== HYPER-PARAMETERS ===========================###

## Here we load the hyperparameters defined above from the dictionary

## Adam
batch_size = config.TRAIN.batch_size  # use 8 if your GPU memory is small, and change [4, 4] in tl.vis.save_images to [2, 4]
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1

## initialize G
n_epoch_init = config.TRAIN.n_epoch_init

## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
shuffle_buffer_size = 128

In [5]:
# create folders to save result images and trained models
save_dir = "samples"
tl.files.exists_or_mkdir(save_dir)
checkpoint_dir = "models"
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples exists ...
[TL] [!] models exists ...


True

In [9]:
# load dataset
train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)

[TL] read 32 from ../srgan/DIV2K_train_HR/
[TL] read 64 from ../srgan/DIV2K_train_HR/
[TL] read 96 from ../srgan/DIV2K_train_HR/
[TL] read 128 from ../srgan/DIV2K_train_HR/
[TL] read 160 from ../srgan/DIV2K_train_HR/
[TL] read 192 from ../srgan/DIV2K_train_HR/
[TL] read 224 from ../srgan/DIV2K_train_HR/
[TL] read 256 from ../srgan/DIV2K_train_HR/
[TL] read 288 from ../srgan/DIV2K_train_HR/
[TL] read 320 from ../srgan/DIV2K_train_HR/
[TL] read 352 from ../srgan/DIV2K_train_HR/
[TL] read 384 from ../srgan/DIV2K_train_HR/
[TL] read 416 from ../srgan/DIV2K_train_HR/
[TL] read 448 from ../srgan/DIV2K_train_HR/
[TL] read 480 from ../srgan/DIV2K_train_HR/
[TL] read 512 from ../srgan/DIV2K_train_HR/
[TL] read 544 from ../srgan/DIV2K_train_HR/
[TL] read 576 from ../srgan/DIV2K_train_HR/
[TL] read 608 from ../srgan/DIV2K_train_HR/
[TL] read 640 from ../srgan/DIV2K_train_HR/
[TL] read 672 from ../srgan/DIV2K_train_HR/
[TL] read 704 from ../srgan/DIV2K_train_HR/
[TL] read 736 from ../srgan/DIV2K_t

In [10]:
def generator_train():
    for img in train_hr_imgs:
        yield img
def _map_fn_train(img):
    hr_patch = tf.image.random_crop(img, [384, 384, 3])
    hr_patch = hr_patch / (255. / 2.)
    hr_patch = hr_patch - 1.
    hr_patch = tf.image.random_flip_left_right(hr_patch)
    lr_patch = tf.image.resize(hr_patch, size=[96, 96])
    return lr_patch, hr_patch

train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
    # train_ds = train_ds.repeat(n_epoch_init + n_epoch)
train_ds = train_ds.shuffle(shuffle_buffer_size)
train_ds = train_ds.prefetch(buffer_size=2)
train_ds = train_ds.batch(batch_size)
    # value = train_ds.make_one_shot_iterator().get_next()

In [11]:
G = get_G((batch_size, 96, 96, 3))
D = get_D((batch_size, 384, 384, 3))
VGG = tl.models.vgg19(pretrained=True, end_with='pool4', mode='static')

lr_v = tf.Variable(lr_init)
g_optimizer_init = tf.optimizers.Adam(lr_v, beta_1=beta1)
g_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)
d_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)

G.train()
D.train()
VGG.train()

[TL] Input  _inputlayer_1: (4, 96, 96, 3)
[TL] Conv2d conv2d_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_3: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_1: fn: add act: No Activation
[TL] Conv2d conv2d_4: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_3: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_5: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_4: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_2

[TL] BatchNorm batchnorm2d_34: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_40: n_filter: 256 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_35: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_41: n_filter: 512 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_36: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_42: n_filter: 1024 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_37: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_43: n_filter: 2048 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_38: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_44: n_filter: 1024 filter_size: (1, 1) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNo

In [12]:
## initialize learning (G)
n_step_epoch = round(n_epoch_init // batch_size)
for epoch in range(n_epoch_init):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape() as tape:
            fake_hr_patchs = G(lr_patchs)
            mse_loss = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True)
        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad, G.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, mse: {:.3f} ".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss))
    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_hr_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_init_{}.png'.format(epoch)))

Epoch: [0/50] step: [0/12] time: 1.664s, mse: 0.442 
Epoch: [0/50] step: [1/12] time: 0.691s, mse: 0.376 
Epoch: [0/50] step: [2/12] time: 0.668s, mse: 0.328 
Epoch: [0/50] step: [3/12] time: 0.427s, mse: 0.440 
Epoch: [0/50] step: [4/12] time: 0.294s, mse: 0.298 
Epoch: [0/50] step: [5/12] time: 0.266s, mse: 0.353 
Epoch: [0/50] step: [6/12] time: 0.272s, mse: 0.299 
Epoch: [0/50] step: [7/12] time: 0.276s, mse: 0.269 
Epoch: [0/50] step: [8/12] time: 0.320s, mse: 0.223 
Epoch: [0/50] step: [9/12] time: 0.221s, mse: 0.285 
Epoch: [0/50] step: [10/12] time: 0.227s, mse: 0.238 
Epoch: [0/50] step: [11/12] time: 0.265s, mse: 0.299 
Epoch: [0/50] step: [12/12] time: 0.239s, mse: 0.282 
Epoch: [0/50] step: [13/12] time: 0.273s, mse: 0.355 
Epoch: [0/50] step: [14/12] time: 0.298s, mse: 0.204 
Epoch: [0/50] step: [15/12] time: 0.292s, mse: 0.254 
Epoch: [0/50] step: [16/12] time: 0.205s, mse: 0.140 
Epoch: [0/50] step: [17/12] time: 0.237s, mse: 0.145 
Epoch: [0/50] step: [18/12] time: 0.23

Epoch: [0/50] step: [151/12] time: 0.162s, mse: 0.065 
Epoch: [0/50] step: [152/12] time: 0.164s, mse: 0.070 
Epoch: [0/50] step: [153/12] time: 0.185s, mse: 0.067 
Epoch: [0/50] step: [154/12] time: 0.164s, mse: 0.055 
Epoch: [0/50] step: [155/12] time: 0.159s, mse: 0.082 
Epoch: [0/50] step: [156/12] time: 0.169s, mse: 0.102 
Epoch: [0/50] step: [157/12] time: 0.172s, mse: 0.037 
Epoch: [0/50] step: [158/12] time: 0.152s, mse: 0.047 
Epoch: [0/50] step: [159/12] time: 0.167s, mse: 0.095 
Epoch: [0/50] step: [160/12] time: 0.171s, mse: 0.039 
Epoch: [0/50] step: [161/12] time: 0.159s, mse: 0.069 
Epoch: [0/50] step: [162/12] time: 0.181s, mse: 0.087 
Epoch: [0/50] step: [163/12] time: 0.147s, mse: 0.058 
Epoch: [0/50] step: [164/12] time: 0.148s, mse: 0.096 
Epoch: [0/50] step: [165/12] time: 0.175s, mse: 0.061 
Epoch: [0/50] step: [166/12] time: 0.159s, mse: 0.081 
Epoch: [0/50] step: [167/12] time: 0.153s, mse: 0.087 
Epoch: [0/50] step: [168/12] time: 0.151s, mse: 0.050 
Epoch: [0/

Epoch: [1/50] step: [102/12] time: 0.319s, mse: 0.039 
Epoch: [1/50] step: [103/12] time: 0.267s, mse: 0.042 
Epoch: [1/50] step: [104/12] time: 0.304s, mse: 0.077 
Epoch: [1/50] step: [105/12] time: 0.233s, mse: 0.045 
Epoch: [1/50] step: [106/12] time: 0.242s, mse: 0.026 
Epoch: [1/50] step: [107/12] time: 0.195s, mse: 0.034 
Epoch: [1/50] step: [108/12] time: 0.307s, mse: 0.050 
Epoch: [1/50] step: [109/12] time: 0.296s, mse: 0.046 
Epoch: [1/50] step: [110/12] time: 0.305s, mse: 0.049 
Epoch: [1/50] step: [111/12] time: 0.257s, mse: 0.034 
Epoch: [1/50] step: [112/12] time: 0.195s, mse: 0.030 
Epoch: [1/50] step: [113/12] time: 0.245s, mse: 0.105 
Epoch: [1/50] step: [114/12] time: 0.285s, mse: 0.044 
Epoch: [1/50] step: [115/12] time: 0.159s, mse: 0.020 
Epoch: [1/50] step: [116/12] time: 0.281s, mse: 0.022 
Epoch: [1/50] step: [117/12] time: 0.203s, mse: 0.034 
Epoch: [1/50] step: [118/12] time: 0.213s, mse: 0.045 
Epoch: [1/50] step: [119/12] time: 0.336s, mse: 0.067 
Epoch: [1/

Epoch: [2/50] step: [53/12] time: 0.244s, mse: 0.032 
Epoch: [2/50] step: [54/12] time: 0.242s, mse: 0.049 
Epoch: [2/50] step: [55/12] time: 0.230s, mse: 0.035 
Epoch: [2/50] step: [56/12] time: 0.237s, mse: 0.032 
Epoch: [2/50] step: [57/12] time: 0.274s, mse: 0.034 
Epoch: [2/50] step: [58/12] time: 0.223s, mse: 0.031 
Epoch: [2/50] step: [59/12] time: 0.300s, mse: 0.043 
Epoch: [2/50] step: [60/12] time: 0.326s, mse: 0.042 
Epoch: [2/50] step: [61/12] time: 0.225s, mse: 0.050 
Epoch: [2/50] step: [62/12] time: 0.279s, mse: 0.018 
Epoch: [2/50] step: [63/12] time: 0.232s, mse: 0.051 
Epoch: [2/50] step: [64/12] time: 0.228s, mse: 0.025 
Epoch: [2/50] step: [65/12] time: 0.242s, mse: 0.024 
Epoch: [2/50] step: [66/12] time: 0.238s, mse: 0.043 
Epoch: [2/50] step: [67/12] time: 0.222s, mse: 0.037 
Epoch: [2/50] step: [68/12] time: 0.244s, mse: 0.033 
Epoch: [2/50] step: [69/12] time: 0.228s, mse: 0.038 
Epoch: [2/50] step: [70/12] time: 0.212s, mse: 0.026 
Epoch: [2/50] step: [71/12] 

Epoch: [3/50] step: [3/12] time: 0.248s, mse: 0.027 
Epoch: [3/50] step: [4/12] time: 0.181s, mse: 0.027 
Epoch: [3/50] step: [5/12] time: 0.235s, mse: 0.023 
Epoch: [3/50] step: [6/12] time: 0.203s, mse: 0.035 
Epoch: [3/50] step: [7/12] time: 0.227s, mse: 0.020 
Epoch: [3/50] step: [8/12] time: 0.224s, mse: 0.052 
Epoch: [3/50] step: [9/12] time: 0.252s, mse: 0.025 
Epoch: [3/50] step: [10/12] time: 0.189s, mse: 0.016 
Epoch: [3/50] step: [11/12] time: 0.254s, mse: 0.037 
Epoch: [3/50] step: [12/12] time: 0.191s, mse: 0.035 
Epoch: [3/50] step: [13/12] time: 0.189s, mse: 0.010 
Epoch: [3/50] step: [14/12] time: 0.304s, mse: 0.035 
Epoch: [3/50] step: [15/12] time: 0.226s, mse: 0.035 
Epoch: [3/50] step: [16/12] time: 0.268s, mse: 0.026 
Epoch: [3/50] step: [17/12] time: 0.244s, mse: 0.042 
Epoch: [3/50] step: [18/12] time: 0.237s, mse: 0.041 
Epoch: [3/50] step: [19/12] time: 0.231s, mse: 0.086 
Epoch: [3/50] step: [20/12] time: 0.214s, mse: 0.013 
Epoch: [3/50] step: [21/12] time: 0

...

Epoch: [46/50] step: [191/12] time: 0.158s, mse: 0.013 
Epoch: [46/50] step: [192/12] time: 0.157s, mse: 0.013 
Epoch: [46/50] step: [193/12] time: 0.188s, mse: 0.015 
Epoch: [46/50] step: [194/12] time: 0.154s, mse: 0.014 
Epoch: [46/50] step: [195/12] time: 0.171s, mse: 0.007 
Epoch: [46/50] step: [196/12] time: 0.159s, mse: 0.019 
Epoch: [46/50] step: [197/12] time: 0.152s, mse: 0.016 
Epoch: [46/50] step: [198/12] time: 0.158s, mse: 0.017 
Epoch: [46/50] step: [199/12] time: 0.158s, mse: 0.010 
Epoch: [47/50] step: [0/12] time: 0.623s, mse: 0.014 
Epoch: [47/50] step: [1/12] time: 0.461s, mse: 0.019 
Epoch: [47/50] step: [2/12] time: 0.496s, mse: 0.011 
Epoch: [47/50] step: [3/12] time: 0.342s, mse: 0.019 
Epoch: [47/50] step: [4/12] time: 0.213s, mse: 0.013 
Epoch: [47/50] step: [5/12] time: 0.237s, mse: 0.015 
Epoch: [47/50] step: [6/12] time: 0.222s, mse: 0.009 
Epoch: [47/50] step: [7/12] time: 0.295s, mse: 0.032 
Epoch: [47/50] step: [8/12] time: 0.287s, mse: 0.006 
Epoch: [47

Epoch: [47/50] step: [140/12] time: 0.258s, mse: 0.028 
Epoch: [47/50] step: [141/12] time: 0.228s, mse: 0.006 
Epoch: [47/50] step: [142/12] time: 0.211s, mse: 0.011 
Epoch: [47/50] step: [143/12] time: 0.225s, mse: 0.005 
Epoch: [47/50] step: [144/12] time: 0.217s, mse: 0.024 
Epoch: [47/50] step: [145/12] time: 0.232s, mse: 0.007 
Epoch: [47/50] step: [146/12] time: 0.191s, mse: 0.019 
Epoch: [47/50] step: [147/12] time: 0.250s, mse: 0.018 
Epoch: [47/50] step: [148/12] time: 0.163s, mse: 0.014 
Epoch: [47/50] step: [149/12] time: 0.167s, mse: 0.025 
Epoch: [47/50] step: [150/12] time: 0.152s, mse: 0.032 
Epoch: [47/50] step: [151/12] time: 0.164s, mse: 0.019 
Epoch: [47/50] step: [152/12] time: 0.169s, mse: 0.020 
Epoch: [47/50] step: [153/12] time: 0.158s, mse: 0.015 
Epoch: [47/50] step: [154/12] time: 0.158s, mse: 0.007 
Epoch: [47/50] step: [155/12] time: 0.157s, mse: 0.012 
Epoch: [47/50] step: [156/12] time: 0.155s, mse: 0.021 
Epoch: [47/50] step: [157/12] time: 0.160s, mse:

Epoch: [48/50] step: [89/12] time: 0.307s, mse: 0.011 
Epoch: [48/50] step: [90/12] time: 0.257s, mse: 0.028 
Epoch: [48/50] step: [91/12] time: 0.261s, mse: 0.013 
Epoch: [48/50] step: [92/12] time: 0.206s, mse: 0.017 
Epoch: [48/50] step: [93/12] time: 0.213s, mse: 0.010 
Epoch: [48/50] step: [94/12] time: 0.288s, mse: 0.014 
Epoch: [48/50] step: [95/12] time: 0.203s, mse: 0.008 
Epoch: [48/50] step: [96/12] time: 0.259s, mse: 0.014 
Epoch: [48/50] step: [97/12] time: 0.254s, mse: 0.006 
Epoch: [48/50] step: [98/12] time: 0.248s, mse: 0.012 
Epoch: [48/50] step: [99/12] time: 0.258s, mse: 0.027 
Epoch: [48/50] step: [100/12] time: 0.247s, mse: 0.022 
Epoch: [48/50] step: [101/12] time: 0.268s, mse: 0.007 
Epoch: [48/50] step: [102/12] time: 0.280s, mse: 0.011 
Epoch: [48/50] step: [103/12] time: 0.303s, mse: 0.010 
Epoch: [48/50] step: [104/12] time: 0.240s, mse: 0.010 
Epoch: [48/50] step: [105/12] time: 0.269s, mse: 0.017 
Epoch: [48/50] step: [106/12] time: 0.199s, mse: 0.033 
Epo

Epoch: [49/50] step: [37/12] time: 0.352s, mse: 0.008 
Epoch: [49/50] step: [38/12] time: 0.253s, mse: 0.021 
Epoch: [49/50] step: [39/12] time: 0.287s, mse: 0.006 
Epoch: [49/50] step: [40/12] time: 0.202s, mse: 0.010 
Epoch: [49/50] step: [41/12] time: 0.297s, mse: 0.013 
Epoch: [49/50] step: [42/12] time: 0.293s, mse: 0.020 
Epoch: [49/50] step: [43/12] time: 0.228s, mse: 0.063 
Epoch: [49/50] step: [44/12] time: 0.239s, mse: 0.017 
Epoch: [49/50] step: [45/12] time: 0.255s, mse: 0.034 
Epoch: [49/50] step: [46/12] time: 0.229s, mse: 0.012 
Epoch: [49/50] step: [47/12] time: 0.222s, mse: 0.013 
Epoch: [49/50] step: [48/12] time: 0.252s, mse: 0.008 
Epoch: [49/50] step: [49/12] time: 0.232s, mse: 0.039 
Epoch: [49/50] step: [50/12] time: 0.274s, mse: 0.008 
Epoch: [49/50] step: [51/12] time: 0.201s, mse: 0.008 
Epoch: [49/50] step: [52/12] time: 0.294s, mse: 0.013 
Epoch: [49/50] step: [53/12] time: 0.224s, mse: 0.022 
Epoch: [49/50] step: [54/12] time: 0.239s, mse: 0.017 
Epoch: [49

Epoch: [49/50] step: [186/12] time: 0.174s, mse: 0.051 
Epoch: [49/50] step: [187/12] time: 0.165s, mse: 0.038 
Epoch: [49/50] step: [188/12] time: 0.179s, mse: 0.014 
Epoch: [49/50] step: [189/12] time: 0.164s, mse: 0.005 
Epoch: [49/50] step: [190/12] time: 0.161s, mse: 0.020 
Epoch: [49/50] step: [191/12] time: 0.185s, mse: 0.022 
Epoch: [49/50] step: [192/12] time: 0.172s, mse: 0.004 
Epoch: [49/50] step: [193/12] time: 0.156s, mse: 0.012 
Epoch: [49/50] step: [194/12] time: 0.197s, mse: 0.017 
Epoch: [49/50] step: [195/12] time: 0.174s, mse: 0.014 
Epoch: [49/50] step: [196/12] time: 0.221s, mse: 0.017 
Epoch: [49/50] step: [197/12] time: 0.160s, mse: 0.014 
Epoch: [49/50] step: [198/12] time: 0.192s, mse: 0.019 
Epoch: [49/50] step: [199/12] time: 0.168s, mse: 0.017 


In [13]:
## adversarial learning (G, D)
n_step_epoch = round(n_epoch // batch_size)
writer = tf.summary.create_file_writer('./log/'+ 'd_loss&g_loss')
for epoch in range(n_epoch):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape(persistent=True) as tape:
            fake_patchs = G(lr_patchs)
            logits_fake = D(fake_patchs)
            logits_real = D(hr_patchs)
            feature_fake = VGG((fake_patchs+1)/2.) # the pre-trained VGG uses the input range of [0, 1]
            feature_real = VGG((hr_patchs+1)/2.)
            d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real))
            d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake))
            d_loss = d_loss1 + d_loss2
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake))
            mse_loss = tl.cost.mean_squared_error(fake_patchs, hr_patchs, is_mean=True)
            vgg_loss = 2e-6 * tl.cost.mean_squared_error(feature_fake, feature_real, is_mean=True)
            g_loss = mse_loss + vgg_loss + g_gan_loss

            if epoch >0:
                with writer.as_default():
                    tf.summary.scalar('d_loss', d_loss, step=epoch+1)
                    tf.summary.scalar('g_loss', g_loss, step=epoch+1)

        grad = tape.gradient(g_loss, G.trainable_weights)
        g_optimizer.apply_gradients(zip(grad, G.trainable_weights))
        grad = tape.gradient(d_loss, D.trainable_weights)
        d_optimizer.apply_gradients(zip(grad, D.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, g_loss(mse:{:.3f}, vgg:{:.3f}, adv:{:.3f}) d_loss: {:.3f}".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss, vgg_loss, g_gan_loss, d_loss))

    # update the learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        lr_v.assign(lr_init * new_lr_decay)
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)

    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_{}.png'.format(epoch)))
        G.save_weights(os.path.join(checkpoint_dir, 'g.h5'))
        D.save_weights(os.path.join(checkpoint_dir, 'd.h5'))

Epoch: [0/50] step: [0/12] time: 2.699s, g_loss(mse:0.011, vgg:0.031, adv:0.000) d_loss: 1.980
Epoch: [0/50] step: [1/12] time: 0.932s, g_loss(mse:0.012, vgg:0.025, adv:0.024) d_loss: 24.212
Epoch: [0/50] step: [2/12] time: 0.862s, g_loss(mse:0.028, vgg:0.041, adv:0.013) d_loss: 12.632
Epoch: [0/50] step: [3/12] time: 0.925s, g_loss(mse:0.013, vgg:0.039, adv:0.001) d_loss: 1.771
Epoch: [0/50] step: [4/12] time: 0.860s, g_loss(mse:0.027, vgg:0.089, adv:0.000) d_loss: 7.407
Epoch: [0/50] step: [5/12] time: 0.894s, g_loss(mse:0.027, vgg:0.056, adv:0.000) d_loss: 7.137
Epoch: [0/50] step: [6/12] time: 0.852s, g_loss(mse:0.020, vgg:0.045, adv:0.000) d_loss: 5.778
Epoch: [0/50] step: [7/12] time: 0.934s, g_loss(mse:0.018, vgg:0.034, adv:0.000) d_loss: 2.012
Epoch: [0/50] step: [8/12] time: 0.965s, g_loss(mse:0.008, vgg:0.029, adv:0.003) d_loss: 3.681
Epoch: [0/50] step: [9/12] time: 0.886s, g_loss(mse:0.009, vgg:0.027, adv:0.003) d_loss: 3.002
Epoch: [0/50] step: [10/12] time: 0.917s, g_loss

Epoch: [0/50] step: [86/12] time: 0.929s, g_loss(mse:0.008, vgg:0.025, adv:0.001) d_loss: 1.405
Epoch: [0/50] step: [87/12] time: 0.914s, g_loss(mse:0.021, vgg:0.037, adv:0.001) d_loss: 1.417
Epoch: [0/50] step: [88/12] time: 0.938s, g_loss(mse:0.009, vgg:0.031, adv:0.001) d_loss: 1.433
Epoch: [0/50] step: [89/12] time: 0.961s, g_loss(mse:0.027, vgg:0.049, adv:0.001) d_loss: 1.439
Epoch: [0/50] step: [90/12] time: 0.943s, g_loss(mse:0.022, vgg:0.041, adv:0.001) d_loss: 1.477
Epoch: [0/50] step: [91/12] time: 0.890s, g_loss(mse:0.007, vgg:0.031, adv:0.000) d_loss: 1.514
Epoch: [0/50] step: [92/12] time: 0.890s, g_loss(mse:0.016, vgg:0.048, adv:0.001) d_loss: 1.395
Epoch: [0/50] step: [93/12] time: 0.980s, g_loss(mse:0.009, vgg:0.035, adv:0.001) d_loss: 1.445
Epoch: [0/50] step: [94/12] time: 0.862s, g_loss(mse:0.014, vgg:0.044, adv:0.001) d_loss: 1.490
Epoch: [0/50] step: [95/12] time: 0.851s, g_loss(mse:0.047, vgg:0.080, adv:0.001) d_loss: 1.431
Epoch: [0/50] step: [96/12] time: 0.872s

Epoch: [0/50] step: [171/12] time: 0.808s, g_loss(mse:0.011, vgg:0.027, adv:0.001) d_loss: 1.513
Epoch: [0/50] step: [172/12] time: 0.852s, g_loss(mse:0.016, vgg:0.024, adv:0.000) d_loss: 1.508
Epoch: [0/50] step: [173/12] time: 0.808s, g_loss(mse:0.041, vgg:0.085, adv:0.001) d_loss: 1.510
Epoch: [0/50] step: [174/12] time: 0.862s, g_loss(mse:0.026, vgg:0.070, adv:0.001) d_loss: 1.414
Epoch: [0/50] step: [175/12] time: 0.890s, g_loss(mse:0.019, vgg:0.052, adv:0.001) d_loss: 1.413
Epoch: [0/50] step: [176/12] time: 0.868s, g_loss(mse:0.029, vgg:0.053, adv:0.001) d_loss: 1.470
Epoch: [0/50] step: [177/12] time: 0.839s, g_loss(mse:0.050, vgg:0.064, adv:0.001) d_loss: 1.597
Epoch: [0/50] step: [178/12] time: 0.797s, g_loss(mse:0.008, vgg:0.029, adv:0.001) d_loss: 1.419
Epoch: [0/50] step: [179/12] time: 0.904s, g_loss(mse:0.023, vgg:0.046, adv:0.001) d_loss: 1.426
Epoch: [0/50] step: [180/12] time: 0.883s, g_loss(mse:0.014, vgg:0.041, adv:0.000) d_loss: 1.590
Epoch: [0/50] step: [181/12] t

Epoch: [1/50] step: [57/12] time: 0.928s, g_loss(mse:0.018, vgg:0.069, adv:0.001) d_loss: 1.417
Epoch: [1/50] step: [58/12] time: 0.935s, g_loss(mse:0.012, vgg:0.033, adv:0.001) d_loss: 1.399
Epoch: [1/50] step: [59/12] time: 1.022s, g_loss(mse:0.018, vgg:0.046, adv:0.001) d_loss: 1.418
Epoch: [1/50] step: [60/12] time: 0.946s, g_loss(mse:0.016, vgg:0.037, adv:0.001) d_loss: 1.403
Epoch: [1/50] step: [61/12] time: 0.967s, g_loss(mse:0.088, vgg:0.121, adv:0.001) d_loss: 1.389
Epoch: [1/50] step: [62/12] time: 1.034s, g_loss(mse:0.014, vgg:0.029, adv:0.001) d_loss: 1.391
Epoch: [1/50] step: [63/12] time: 0.866s, g_loss(mse:0.020, vgg:0.066, adv:0.001) d_loss: 1.394
Epoch: [1/50] step: [64/12] time: 0.922s, g_loss(mse:0.007, vgg:0.028, adv:0.001) d_loss: 1.396
Epoch: [1/50] step: [65/12] time: 0.880s, g_loss(mse:0.016, vgg:0.058, adv:0.001) d_loss: 1.394
Epoch: [1/50] step: [66/12] time: 0.983s, g_loss(mse:0.029, vgg:0.031, adv:0.001) d_loss: 1.392
Epoch: [1/50] step: [67/12] time: 0.918s

Epoch: [1/50] step: [142/12] time: 0.912s, g_loss(mse:0.036, vgg:0.050, adv:0.001) d_loss: 1.424
Epoch: [1/50] step: [143/12] time: 0.959s, g_loss(mse:0.008, vgg:0.025, adv:0.001) d_loss: 1.398
Epoch: [1/50] step: [144/12] time: 0.889s, g_loss(mse:0.010, vgg:0.024, adv:0.001) d_loss: 1.416
Epoch: [1/50] step: [145/12] time: 0.905s, g_loss(mse:0.017, vgg:0.060, adv:0.001) d_loss: 1.422
Epoch: [1/50] step: [146/12] time: 0.898s, g_loss(mse:0.011, vgg:0.037, adv:0.001) d_loss: 1.398
Epoch: [1/50] step: [147/12] time: 0.860s, g_loss(mse:0.017, vgg:0.054, adv:0.001) d_loss: 1.397
Epoch: [1/50] step: [148/12] time: 0.839s, g_loss(mse:0.005, vgg:0.024, adv:0.001) d_loss: 1.388
Epoch: [1/50] step: [149/12] time: 0.830s, g_loss(mse:0.015, vgg:0.054, adv:0.001) d_loss: 1.401
Epoch: [1/50] step: [150/12] time: 0.815s, g_loss(mse:0.009, vgg:0.023, adv:0.001) d_loss: 1.403
Epoch: [1/50] step: [151/12] time: 0.875s, g_loss(mse:0.016, vgg:0.050, adv:0.001) d_loss: 1.397
Epoch: [1/50] step: [152/12] t

...

Epoch: [48/50] step: [56/12] time: 0.964s, g_loss(mse:0.021, vgg:0.065, adv:0.005) d_loss: 0.010
Epoch: [48/50] step: [57/12] time: 0.877s, g_loss(mse:0.021, vgg:0.037, adv:0.005) d_loss: 0.012
Epoch: [48/50] step: [58/12] time: 0.873s, g_loss(mse:0.011, vgg:0.027, adv:0.002) d_loss: 0.118
Epoch: [48/50] step: [59/12] time: 0.912s, g_loss(mse:0.018, vgg:0.023, adv:0.005) d_loss: 0.009
Epoch: [48/50] step: [60/12] time: 0.912s, g_loss(mse:0.003, vgg:0.016, adv:0.002) d_loss: 0.211
Epoch: [48/50] step: [61/12] time: 0.879s, g_loss(mse:0.016, vgg:0.038, adv:0.004) d_loss: 0.016
Epoch: [48/50] step: [62/12] time: 0.914s, g_loss(mse:0.009, vgg:0.032, adv:0.002) d_loss: 0.233
Epoch: [48/50] step: [63/12] time: 0.870s, g_loss(mse:0.040, vgg:0.084, adv:0.005) d_loss: 0.007
Epoch: [48/50] step: [64/12] time: 0.874s, g_loss(mse:0.013, vgg:0.030, adv:0.004) d_loss: 0.012
Epoch: [48/50] step: [65/12] time: 0.867s, g_loss(mse:0.017, vgg:0.049, adv:0.004) d_loss: 0.018
Epoch: [48/50] step: [66/12] t

Epoch: [48/50] step: [141/12] time: 0.917s, g_loss(mse:0.014, vgg:0.030, adv:0.005) d_loss: 0.008
Epoch: [48/50] step: [142/12] time: 0.993s, g_loss(mse:0.006, vgg:0.024, adv:0.003) d_loss: 0.097
Epoch: [48/50] step: [143/12] time: 0.884s, g_loss(mse:0.029, vgg:0.057, adv:0.005) d_loss: 0.006
Epoch: [48/50] step: [144/12] time: 0.850s, g_loss(mse:0.017, vgg:0.045, adv:0.005) d_loss: 0.005
Epoch: [48/50] step: [145/12] time: 0.922s, g_loss(mse:0.019, vgg:0.052, adv:0.005) d_loss: 0.009
Epoch: [48/50] step: [146/12] time: 0.875s, g_loss(mse:0.010, vgg:0.048, adv:0.005) d_loss: 0.006
Epoch: [48/50] step: [147/12] time: 0.886s, g_loss(mse:0.033, vgg:0.060, adv:0.006) d_loss: 0.004
Epoch: [48/50] step: [148/12] time: 0.818s, g_loss(mse:0.032, vgg:0.037, adv:0.005) d_loss: 0.005
Epoch: [48/50] step: [149/12] time: 0.831s, g_loss(mse:0.004, vgg:0.018, adv:0.005) d_loss: 0.031
Epoch: [48/50] step: [150/12] time: 0.814s, g_loss(mse:0.007, vgg:0.021, adv:0.004) d_loss: 0.026
Epoch: [48/50] step:

Epoch: [49/50] step: [25/12] time: 0.855s, g_loss(mse:0.018, vgg:0.039, adv:0.002) d_loss: 0.130
Epoch: [49/50] step: [26/12] time: 0.860s, g_loss(mse:0.018, vgg:0.035, adv:0.001) d_loss: 0.816
Epoch: [49/50] step: [27/12] time: 0.868s, g_loss(mse:0.024, vgg:0.047, adv:0.006) d_loss: 0.005
Epoch: [49/50] step: [28/12] time: 0.884s, g_loss(mse:0.015, vgg:0.039, adv:0.005) d_loss: 0.018
Epoch: [49/50] step: [29/12] time: 0.878s, g_loss(mse:0.010, vgg:0.035, adv:0.005) d_loss: 1.180
Epoch: [49/50] step: [30/12] time: 0.894s, g_loss(mse:0.011, vgg:0.015, adv:0.000) d_loss: 2.851
Epoch: [49/50] step: [31/12] time: 0.916s, g_loss(mse:0.018, vgg:0.024, adv:0.003) d_loss: 0.045
Epoch: [49/50] step: [32/12] time: 0.861s, g_loss(mse:0.011, vgg:0.041, adv:0.003) d_loss: 0.514
Epoch: [49/50] step: [33/12] time: 0.978s, g_loss(mse:0.016, vgg:0.039, adv:0.006) d_loss: 0.113
Epoch: [49/50] step: [34/12] time: 0.908s, g_loss(mse:0.018, vgg:0.040, adv:0.004) d_loss: 0.018
Epoch: [49/50] step: [35/12] t

Epoch: [49/50] step: [110/12] time: 0.916s, g_loss(mse:0.010, vgg:0.031, adv:0.003) d_loss: 1.174
Epoch: [49/50] step: [111/12] time: 0.938s, g_loss(mse:0.017, vgg:0.051, adv:0.004) d_loss: 0.053
Epoch: [49/50] step: [112/12] time: 0.904s, g_loss(mse:0.014, vgg:0.028, adv:0.003) d_loss: 0.094
Epoch: [49/50] step: [113/12] time: 0.895s, g_loss(mse:0.032, vgg:0.053, adv:0.004) d_loss: 0.023
Epoch: [49/50] step: [114/12] time: 0.931s, g_loss(mse:0.008, vgg:0.028, adv:0.002) d_loss: 0.716
Epoch: [49/50] step: [115/12] time: 0.945s, g_loss(mse:0.027, vgg:0.054, adv:0.002) d_loss: 0.111
Epoch: [49/50] step: [116/12] time: 0.918s, g_loss(mse:0.011, vgg:0.026, adv:0.001) d_loss: 0.459
Epoch: [49/50] step: [117/12] time: 0.948s, g_loss(mse:0.024, vgg:0.067, adv:0.004) d_loss: 0.024
Epoch: [49/50] step: [118/12] time: 0.888s, g_loss(mse:0.016, vgg:0.039, adv:0.003) d_loss: 0.050
Epoch: [49/50] step: [119/12] time: 0.912s, g_loss(mse:0.016, vgg:0.038, adv:0.003) d_loss: 0.046
Epoch: [49/50] step:

Epoch: [49/50] step: [194/12] time: 0.859s, g_loss(mse:0.037, vgg:0.050, adv:0.005) d_loss: 0.023
Epoch: [49/50] step: [195/12] time: 0.802s, g_loss(mse:0.015, vgg:0.038, adv:0.003) d_loss: 0.089
Epoch: [49/50] step: [196/12] time: 0.862s, g_loss(mse:0.011, vgg:0.037, adv:0.004) d_loss: 1.185
Epoch: [49/50] step: [197/12] time: 0.849s, g_loss(mse:0.041, vgg:0.080, adv:0.004) d_loss: 0.014
Epoch: [49/50] step: [198/12] time: 0.850s, g_loss(mse:0.023, vgg:0.048, adv:0.004) d_loss: 0.023
Epoch: [49/50] step: [199/12] time: 0.845s, g_loss(mse:0.013, vgg:0.032, adv:0.004) d_loss: 0.260


In [6]:
###====================== Loading Test Data ===========================###
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))
valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, path=config.VALID.lr_img_path, n_threads=32)
valid_hr_imgs = tl.vis.read_images(valid_hr_img_list, path=config.VALID.hr_img_path, n_threads=32)

###========================== DEFINE MODEL ============================###
imid = 0  # 0: Penguin  81: Butterfly  53: Bird  64: Castle
valid_lr_img = valid_lr_imgs[imid]
valid_hr_img = valid_hr_imgs[imid]
# valid_lr_img = get_imgs_fn('test.png', 'data2017/')  # if you want to test your own image
valid_lr_img = (valid_lr_img / 127.5) - 1  # rescale to [-1, 1]
# print(valid_lr_img.min(), valid_lr_img.max())

G2 = get_Gval([1, None, None, 3])
G2.load_weights(os.path.join(checkpoint_dir, 'g.h5'))
G2.eval()



[TL] read 32 from ../srgan/DIV2K_valid_LR_bicubic/X4/
[TL] read 64 from ../srgan/DIV2K_valid_LR_bicubic/X4/
[TL] read 96 from ../srgan/DIV2K_valid_LR_bicubic/X4/
[TL] read 100 from ../srgan/DIV2K_valid_LR_bicubic/X4/
[TL] read 32 from ../srgan/DIV2K_valid_HR/
[TL] read 64 from ../srgan/DIV2K_valid_HR/
[TL] read 96 from ../srgan/DIV2K_valid_HR/
[TL] read 100 from ../srgan/DIV2K_valid_HR/
[TL] Input  _inputlayer_1: [1, None, None, 3]
[TL] Conv2d conv2d_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_3: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_1: fn: add act: No Activation
[TL] Conv2d conv2d_4: n_f

[TL] SubpixelConv2d  subpixelconv2d_2: scale: 2 act: relu
[TL] Conv2d conv2d_37: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh
[TL] [*] Load models/g.h5 SUCCESS!


In [14]:
#valid_hr_img =(valid_hr_img / 127.5) - 1
# valid_hr_img = np.asarray(valid_hr_img, dtype=np.float32)
# valid_hr_img = valid_hr_img[np.newaxis,:,:,:]
valid_lr_img = valid_lr_img[0]

In [15]:
valid_lr_img = np.asarray(valid_lr_img, dtype=np.float32)
size = [valid_lr_img.shape[1], valid_lr_img.shape[2]]

output = G2(valid_lr_img).numpy()
psnr = tf.image.psnr(output, valid_hr_img, max_val=1.0)
print(psnr)
ssim2 = tf.image.ssim(output, valid_hr_img, max_val=1.0, filter_size=11,
              filter_sigma=1.5, k1=0.01, k2=0.03)
print(ssim2)
mse = tf.keras.losses.mean_squared_error(output,valid_hr_img).numpy()


print(np.sum(mse)/(mse.size))

tf.Tensor([21.113031], shape=(1,), dtype=float32)


AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'